In [1]:
import torch

In [2]:
import torchtext

In [3]:
from torchtext.datasets import text_classification

In [4]:
NGRAMS = 2

In [5]:
import os

In [6]:
if not os.path.isdir('./.data'):
    os.mkdir('./.data')

In [7]:
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](root = './.data', ngrams= NGRAMS, vocab=None)

120000lines [00:07, 15956.27lines/s]
120000lines [00:13, 8901.53lines/s]
7600lines [00:00, 8627.02lines/s]


In [8]:
BATCH_SIZE = 16

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [11]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [13]:
model.embedding.weight[10:15]

tensor([[ 0.0651,  0.0304, -0.4281,  0.1517, -0.2144,  0.4823, -0.4475, -0.3091,
          0.4425,  0.1859, -0.2278, -0.2926,  0.3620,  0.1438,  0.1043,  0.2641,
          0.0581,  0.2177,  0.2984, -0.3495, -0.0938,  0.3112, -0.3127, -0.0838,
          0.3244, -0.4936, -0.2568, -0.3273, -0.4760,  0.0677,  0.3018,  0.1236],
        [ 0.0107, -0.1493,  0.2340,  0.3049,  0.4198, -0.4674, -0.2883,  0.2055,
          0.0704,  0.4376,  0.1954,  0.4294,  0.2506,  0.3782,  0.3937,  0.0384,
         -0.3007,  0.4602, -0.1032, -0.2604,  0.3558, -0.1598,  0.4931,  0.1770,
          0.2505,  0.1953,  0.2748, -0.0729,  0.2532, -0.4056, -0.1494, -0.4426],
        [ 0.4017,  0.0999, -0.1111,  0.0351,  0.4698, -0.4457, -0.3186, -0.2315,
          0.0316, -0.0769,  0.3937, -0.4664,  0.0535,  0.0991, -0.3025,  0.0708,
          0.1699, -0.0392, -0.2277, -0.1663,  0.2584,  0.1826,  0.2628,  0.1041,
         -0.4986, -0.2929, -0.0061,  0.2314,  0.2687,  0.3857,  0.0426, -0.1814],
        [ 0.2830, -0.2744

In [19]:
iterCount = 0
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    global iterCount
    text = [entry[1] for entry in batch]
    length = [len(entry[1]) for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    iterCount = iterCount + 1
    return text, offsets, label

In [20]:
from torch.utils.data import DataLoader


In [21]:
data = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,collate_fn=generate_batch)

In [22]:
def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)


In [23]:
def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [24]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)
    
    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')
    print(model.embedding.weight[10:12])

Epoch: 1  | time in 0 minutes, 49 seconds
	Loss: 0.0040(train)	|	Acc: 98.2%(train)
	Loss: 0.0001(valid)	|	Acc: 96.7%(valid)
tensor([[ 0.0435, -0.0578, -0.3349,  0.1859, -0.0533,  0.3975, -0.4846, -0.1683,
          0.2210,  0.0940, -0.1398, -0.1770,  0.1249,  0.0708,  0.2275,  0.4073,
          0.1229,  0.2380,  0.0412, -0.4892, -0.0559,  0.2879, -0.2088, -0.1338,
          0.4013, -0.4712, -0.3538, -0.3588, -0.3056, -0.0995,  0.2255, -0.0351],
        [-0.0286, -0.1249, -0.1219, -0.0107,  0.0507, -0.4679,  0.0181,  0.0574,
          0.2048,  0.7144, -0.1072,  0.3459,  0.5389,  0.3681,  0.2995, -0.1708,
         -0.3524,  0.4100,  0.1525, -0.2202,  0.2646, -0.2490,  0.3035,  0.0329,
          0.0099,  0.3081,  0.1874,  0.1801,  0.1376, -0.2061,  0.2695, -0.0115]],
       grad_fn=<SliceBackward>)
Epoch: 2  | time in 0 minutes, 49 seconds
	Loss: 0.0022(train)	|	Acc: 99.0%(train)
	Loss: 0.0001(valid)	|	Acc: 97.4%(valid)
tensor([[-0.0390, -0.0774, -0.3083,  0.2251,  0.0254,  0.3444, -0.541

In [38]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_dataset)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0003(test)	|	Acc: 87.4%(test)


In [39]:
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])

This is a Sports news
